## PDE 1 - 2D


#### Problem Setup

$\phi u + u_{x} + u_{y,y} = f(x,y)$

For the generation of our initial data samples we use:

$\phi = 2$ <br>
$u: \mathbb{R}^2 \rightarrow \mathbb{R}, \; u(x,y) = x^2 + y$ <br>
$f: \mathbb{R}^2 \rightarrow \mathbb{R}, \;f(x,y) = 2(x^2 + x + y)$ <br>
$X_i := (x_i, y_i) \in [0,1] \times [0,1] \in \mathbb{R}^2$ for $i \in \{1, \dotsc, n\}$ 

and our known function values will be $\{u(x_i,y_i), f(x_i,y_i)\}_{i \in \{1, \dotsc, n\}}$.

We assume that $u$ can be represented as a Gaussian process with Matérn kernel, where $\nu = 5/2$.

$u \sim \mathcal{GP}(0, k_{uu}(X_i, X_j; \theta))$, where $\theta = \{\sigma, l_x, l_y\}$.

Set the linear operator to:

$\mathcal{L}_X^{\phi} := \phi + \partial_x + \partial_{y,y}$

so that

$\mathcal{L}_X^{\phi} u = f$

Problem at hand: Estimate $\phi$ (we expect $\phi = 2$).


#### Step 1: Simulate data

In [38]:
import time
import numpy as np
import sympy as sp
from scipy.linalg import solve_triangular
import scipy.optimize as opt

In [39]:
# Global variables: x, y, n, y_u, y_f, s

*Parameters, that can be modified:*

In [40]:
# Number of data samples:
n = 10

# Noise of our data:
s = 0

# Circumventing evaluations of kernel-derivatives at zero:
corr_nan = 1e-4

In [41]:
def simulate_data():
    x = np.random.rand(n)
    y = np.random.rand(n)
    y_u = np.multiply(x,x) + y
    y_f = 2*(np.multiply(x,x) + x + y)
    return (x,y,y_u,y_f)
(x,y,y_u,y_f) = simulate_data()

#### Step 2: Evaluate kernels

$k_{uu}(X_i, X_j; \theta) = \sigma \left( 1+ \sqrt{5}r_l + \frac{5}{3}r_l^2 \right) \exp \left( -\sqrt{5}r_l \right)$, where:

$r_l = \sqrt{\frac{1}{l_x^2}(x_i-x_j)^2 + \frac{1}{l_y^2}(y_i-y_j)^2}$

In [42]:
x_i, x_j, y_i, y_j, sigma, l_x, l_y, phi = sp.symbols('x_i x_j y_i y_j sigma l_x l_y phi')
# kuu_sym = sigma*sp.exp(-1/(2*l_x)*((x_i - x_j)**2) - 1/(2*l_y)*((y_i - y_j)**2))
r_l = sp.sqrt((x_i - x_j)**2/l_x + (y_i - y_j)**2/l_y)
kuu_sym = sigma*(1 + sp.sqrt(5)*r_l + 5/3*r_l**2)*sp.exp(-sp.sqrt(5)*r_l)
kuu_fn = sp.lambdify((x_i, x_j, y_i, y_j, sigma, l_x, l_y), kuu_sym, "numpy")
def kuu(x, y, sigma, l_x, l_y):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kuu_fn(x[i], x[j], y[i], y[j], sigma, l_x, l_y)
    return k

$k_{ff}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} \mathcal{L}_{X_j}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= \phi^2k_{uu} + \phi \frac{\partial}{\partial x_i}k_{uu} + \phi \frac{\partial^2}{\partial y_i^2}k_{uu} + \phi \frac{\partial}{\partial x_j}k_{uu} + \frac{\partial^2}{\partial x_i, x_j}k_{uu} + \frac{\partial^3}{\partial y_i^2 \partial x_j}k_{uu} + \phi \frac{\partial^2}{\partial y_j^2}k_{uu} + \frac{\partial^3}{\partial x_i \partial y_j^2}k_{uu} + \frac{\partial^4}{\partial y_i^2 \partial y_j^2}k_{uu}$

In [43]:
kff_sym = phi**2*kuu_sym \
        + phi*sp.diff(kuu_sym, x_i) \
        + phi*sp.diff(kuu_sym, y_i, y_i) \
        + phi*sp.diff(kuu_sym, x_j) \
        + sp.diff(kuu_sym, x_i, x_j) \
        + sp.diff(kuu_sym, y_i, y_i, x_j) \
        + phi*sp.diff(kuu_sym, y_j, y_j) \
        + sp.diff(kuu_sym, x_i, y_j, y_j) \
        + sp.diff(kuu_sym, y_i, y_i, y_j, y_j)
kff_fn = sp.lambdify((x_i, x_j, y_i, y_j, sigma, l_x, l_y, phi), kff_sym, "numpy")
def kff(x, y, sigma, l_x, l_y, phi):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            if i == j:
                k[i,j] = kff_fn(x[i], x[j] + corr_nan, y[i], y[j] + corr_nan, sigma, l_x, l_y, phi)
            else:
                k[i,j] = kff_fn(x[i], x[j], y[i], y[j], sigma, l_x, l_y, phi)
    return k

$k_{fu}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= \phi k_{uu} + \frac{\partial}{\partial x_i}k_{uu} + \frac{\partial^2}{\partial y_i^2}k_{uu}$

In [44]:
kfu_sym = phi*kuu_sym \
        + sp.diff(kuu_sym, x_i) \
        + sp.diff(kuu_sym, y_i, y_i)
kfu_fn = sp.lambdify((x_i, x_j, y_i, y_j, sigma, l_x, l_y, phi), kfu_sym, "numpy")
def kfu(x, y, sigma, l_x, l_y, phi):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            if i == j:
                k[i,j] = kfu_fn(x[i], x[j] + corr_nan, y[i], y[j] + corr_nan, sigma, l_x, l_y, phi)
            else:
                k[i,j] = kfu_fn(x[i], x[j], y[i], y[j], sigma, l_x, l_y, phi)
    return k

In [45]:
def kuf(x, y, sigma, l_x, l_y, phi):
    return kfu(x, y, sigma, l_x, l_y, phi).T

#### Step 3: Computing the negative log-likelihood (with block matrix inversion, Cholesky decomposition, potentially SVD)

We use the block-inversion technique: Let
$ K = \begin{pmatrix} K_{uu} & K_{uf} \\ K_{fu} & K_{ff} \end{pmatrix} = \begin{pmatrix} A & B \\ B^T & C \end{pmatrix}$. 

Then $det(K) = det(A) det(C-B^T A^{-1} B)$.

$K^{-1} = \begin{pmatrix} A^{-1} + A^{-1} B(C-B^T A^{-1} B)^{-1}B^T A^{-1} & -A^{-1}B(C-B^T A^{-1} B)^{-1} \\
            -(C - B^T A^{-1}B)^{-1}B^T A^{-1} & (C-B^T A^{-1} B)^{-1} \end{pmatrix}$
            
So it suffices to invert $A$ and $C-B^T A^{-1} B$.

A theorem about Schur-complements ensures that $K$ positive-definite implies the positive-definiteness of $K/A = C-B^T A^{-1} B$ as well, so Cholesky should work as well.

In [46]:
def nlml(params):
    
    sigma_exp = np.exp(params[0])
    l_x_exp = np.exp(params[1])
    l_y_exp = np.exp(params[2])
    # phi = params[3]
    
    A = kuu(x, y, sigma_exp, l_x_exp, l_y_exp) + s*np.eye(n)
    B = kfu(x, y, sigma_exp, l_x_exp, l_y_exp, params[3]).T
    C = kff(x, y, sigma_exp, l_x_exp, l_y_exp, params[3]) + s*np.eye(n)
    
    # Inversion of A
    A_inv = np.zeros((n, n))
    
    try:
        L = np.linalg.cholesky(A)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost over np.linalg.inv
        A_inv = L_inv.T @ L_inv
        logdet_A = 2*np.log(np.abs(np.diag(L))).sum()
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(A)
        A_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        logdet_A = np.log(s_mat).sum()
        
    # Inversion of $C-B^T A^{-1} B$
    KA_inv = np.zeros((n, n))
    KA = C - B.T @ A_inv @ B
    
    try:
        L = np.linalg.cholesky(KA)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost over np.linalg.inv
        KA_inv = L_inv.T @ L_inv
        logdet_KA = 2*np.log(np.abs(np.diag(L))).sum()
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(KA)
        KA_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        logdet_KA = np.log(s_mat).sum()
        
    # Piecing it together
    T = A_inv @ B @ KA_inv
    yKy = y_u @ (A_inv + T @ B.T @ A_inv) @ y_u - 2*y_u @ T @ y_f + y_f @ KA_inv @ y_f
    
    return (yKy + logdet_A + logdet_KA)

#### Step 4: Optimize hyperparameters

**1. Nelder-Mead**

In [47]:
Nfeval = 1
def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f}   {2: 3.6f}   {3: 3.6f}   {4: 3.6f}'.format(Nfeval, Xi[0], Xi[1], Xi[2], Xi[3]))
    Nfeval += 1

In [48]:
t0 = time.time()
m_n = opt.minimize(nlml, np.random.rand(4), method="Nelder-Mead", callback = callbackF,
                                        options={'maxfev':5000, 'fatol':0.001, 'xatol':0.001})
t_Nelder = time.time() - t0

   1    0.877706    0.370521    0.823986    0.069210
   2    0.877706    0.370521    0.823986    0.069210
   3    0.865516    0.392283    0.872383    0.064103
   4    0.783231    0.369544    0.931018    0.068819
   5    0.783231    0.369544    0.931018    0.068819
   6    0.605328    0.400322    0.983331    0.064223
   7    0.498567    0.427137    1.112767    0.061317
   8    0.498567    0.427137    1.112767    0.061317
   9    0.498567    0.427137    1.112767    0.061317
  10    0.498567    0.427137    1.112767    0.061317
  11    0.498567    0.427137    1.112767    0.061317
  12    0.540742    0.400727    1.075489    0.065039
  13    0.438877    0.410803    1.156715    0.064148
  14    0.515687    0.424925    1.204715    0.053933
  15    0.515687    0.424925    1.204715    0.053933
  16    0.557676    0.393130    1.185696    0.059990
  17    0.557676    0.393130    1.185696    0.059990
  18    0.738796    0.432306    1.253245    0.043069
  19    0.581770    0.404576    1.450291    0.

 158    1.876351    3.173517    5.035871    1.993079
 159    1.876351    3.173517    5.035871    1.993079
 160    1.876351    3.173517    5.035871    1.993079
 161    1.876351    3.173517    5.035871    1.993079
 162    1.876351    3.173517    5.035871    1.993079
 163    1.876351    3.173517    5.035871    1.993079
 164    1.876351    3.173517    5.035871    1.993079
 165    1.876351    3.173517    5.035871    1.993079
 166    1.876351    3.173517    5.035871    1.993079
 167    1.876351    3.173517    5.035871    1.993079
 168    1.876351    3.173517    5.035871    1.993079
 169    1.876351    3.173517    5.035871    1.993079
 170    1.876351    3.173517    5.035871    1.993079
 171    1.876351    3.173517    5.035871    1.993079
 172    1.876351    3.173517    5.035871    1.993079
 173    1.876351    3.173517    5.035871    1.993079
 174    1.876351    3.173517    5.035871    1.993079
 175    1.876351    3.173517    5.035871    1.993079
 176    1.876351    3.173517    5.035871    1.

 313    1.876351    3.173517    5.035871    1.993079
 314    1.876351    3.173517    5.035871    1.993079
 315    1.876351    3.173517    5.035871    1.993079
 316    1.876351    3.173517    5.035871    1.993079
 317    1.876351    3.173517    5.035871    1.993079
 318    1.876351    3.173517    5.035871    1.993079
 319    1.876351    3.173517    5.035871    1.993079
 320    1.876351    3.173517    5.035871    1.993079
 321    1.876351    3.173517    5.035871    1.993079
 322    1.876351    3.173517    5.035871    1.993079
 323    1.876351    3.173517    5.035871    1.993079
 324    1.876351    3.173517    5.035871    1.993079
 325    1.876351    3.173517    5.035871    1.993079
 326    1.876351    3.173517    5.035871    1.993079
 327    1.876351    3.173517    5.035871    1.993079
 328    1.876351    3.173517    5.035871    1.993079
 329    1.876351    3.173517    5.035871    1.993079
 330    1.876351    3.173517    5.035871    1.993079
 331    1.876351    3.173517    5.035871    1.

 468    1.876351    3.173517    5.035871    1.993079
 469    1.876351    3.173517    5.035871    1.993079
 470    1.876351    3.173517    5.035871    1.993079
 471    1.876351    3.173517    5.035871    1.993079
 472    1.876351    3.173517    5.035871    1.993079
 473    1.876351    3.173517    5.035871    1.993079
 474    1.876351    3.173517    5.035871    1.993079
 475    1.876351    3.173517    5.035871    1.993079
 476    1.876351    3.173517    5.035871    1.993079
 477    1.876351    3.173517    5.035871    1.993079
 478    1.876351    3.173517    5.035871    1.993079
 479    1.876351    3.173517    5.035871    1.993079
 480    1.876351    3.173517    5.035871    1.993079
 481    1.876351    3.173517    5.035871    1.993079
 482    1.876351    3.173517    5.035871    1.993079
 483    1.876351    3.173517    5.035871    1.993079
 484    1.876351    3.173517    5.035871    1.993079
 485    1.876351    3.173517    5.035871    1.993079
 486    1.876351    3.173517    5.035871    1.

 623    1.876351    3.173517    5.035871    1.993079
 624    1.876351    3.173517    5.035871    1.993079
 625    1.876351    3.173517    5.035871    1.993079
 626    1.876351    3.173517    5.035871    1.993079
 627    1.876351    3.173517    5.035871    1.993079
 628    1.876351    3.173517    5.035871    1.993079
 629    1.876351    3.173517    5.035871    1.993079
 630    1.876351    3.173517    5.035871    1.993079
 631    1.876351    3.173517    5.035871    1.993079
 632    1.876351    3.173517    5.035871    1.993079
 633    1.876351    3.173517    5.035871    1.993079
 634    1.876351    3.173517    5.035871    1.993079
 635    1.876351    3.173517    5.035871    1.993079
 636    1.876351    3.173517    5.035871    1.993079
 637    1.876351    3.173517    5.035871    1.993079
 638    1.876351    3.173517    5.035871    1.993079
 639    1.876351    3.173517    5.035871    1.993079
 640    1.876351    3.173517    5.035871    1.993079
 641    1.876351    3.173517    5.035871    1.

 778    1.876351    3.173517    5.035871    1.993079
 779    1.876351    3.173517    5.035871    1.993079
 780    1.876351    3.173517    5.035871    1.993079
 781    1.876351    3.173517    5.035871    1.993079
 782    1.876351    3.173517    5.035871    1.993079
 783    1.876351    3.173517    5.035871    1.993079
 784    1.876351    3.173517    5.035871    1.993079
 785    1.876351    3.173517    5.035871    1.993079
 786    1.876351    3.173517    5.035871    1.993079
 787    1.876351    3.173517    5.035871    1.993079
 788    1.876351    3.173517    5.035871    1.993079
 789    1.876351    3.173517    5.035871    1.993079
 790    1.876351    3.173517    5.035871    1.993079
 791    1.876351    3.173517    5.035871    1.993079
 792    1.876351    3.173517    5.035871    1.993079
 793    1.876351    3.173517    5.035871    1.993079
 794    1.876351    3.173517    5.035871    1.993079
 795    1.876351    3.173517    5.035871    1.993079
 796    1.876351    3.173517    5.035871    1.

 933    1.876351    3.173517    5.035871    1.993079
 934    1.876351    3.173517    5.035871    1.993079
 935    1.876351    3.173517    5.035871    1.993079
 936    1.876351    3.173517    5.035871    1.993079
 937    1.876351    3.173517    5.035871    1.993079
 938    1.876351    3.173517    5.035871    1.993079
 939    1.876351    3.173517    5.035871    1.993079
 940    1.876351    3.173517    5.035871    1.993079
 941    1.876351    3.173517    5.035871    1.993079
 942    1.876351    3.173517    5.035871    1.993079
 943    1.876351    3.173517    5.035871    1.993079
 944    1.876351    3.173517    5.035871    1.993079
 945    1.876351    3.173517    5.035871    1.993079
 946    1.876351    3.173517    5.035871    1.993079
 947    1.876351    3.173517    5.035871    1.993079
 948    1.876351    3.173517    5.035871    1.993079
 949    1.876351    3.173517    5.035871    1.993079
 950    1.876351    3.173517    5.035871    1.993079
 951    1.876351    3.173517    5.035871    1.

In [49]:
print('Inferred parameter: %.4f' % m_n.x[3])

Inferred parameter: 1.9931


In [50]:
print('Time: %d seconds' % t_Nelder)

Time: 362 seconds
